<a href="https://colab.research.google.com/github/Private-Detective/CloudStream/blob/main/XL8R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src='https://www.libtorrent.org/img/logo-color-text.png' height='38'/>

In [ ]:
from IPython.display import clear_output
!pip install libtorrent
clear_output()
import libtorrent as lt
import time
import os
import sys

save_path = os.path.join(os.getcwd(), 'drive/MyDrive/Torrents')
magnet_link = "" #@param {type:"string"}

ses = lt.session()

try:
    params = lt.parse_magnet_uri(magnet_link)
    params.save_path = save_path
    handle = ses.add_torrent(params)

    start_time = time.time()

    while not handle.status().is_seeding:
        elapsed_time = time.time() - start_time
        if handle.status().total_wanted_done > 0:
            remaining_time = (handle.status().total_wanted - handle.status().total_wanted_done) / (handle.status().total_wanted_done / elapsed_time)
            remaining_time_str = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
        else:
            remaining_time_str = "Unknown"

        print(f"\r{handle.status().name[:15]}.. : {handle.status().total_wanted_done / (1024 * 1024):.2f}MB [{elapsed_time:.0f} seconds, {(handle.status().total_wanted_done / elapsed_time) / (1024 * 1024):.2f}MB/s] Remaining time: {remaining_time_str}", end=' ')
        sys.stdout.flush()

        time.sleep(1)

    if handle.status().is_seeding:
        print("\nTorrent Download Completed!")

except Exception as e:
    print(f"Error: {e}")

# <img src='https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_RGB_Green.png' height="39"/>

In [ ]:

from IPython.display import clear_output

# Check if spotdl is installed
SPOTDL = !(command -v "spotdl")
if not SPOTDL:
    !pip install spotdl
    clear_output()

# Add search query option
search_query = ""  # @param {type:"string"}
link = ""  # @param {type:"string"}
output = "/content/{title} - {artists}"  # kept the original output format

options = [
    "--format mp3",
    "--bitrate 128k",  # 128k as default bitrate
    "--lyrics musixmatch",  # Musixmatch as default lyrics provider
    "--sponsor-block",
    "--generate-lrc",
    "--print-errors",
    f"--output \"{output}\""
]

# Add link to spotdl command if provided
if link:
    !spotdl download {link} {" ".join(options)}
else:
    !spotdl download "{search_query}" {" ".join(options)}

try:
    # No need for eval or exec, just execute the command directly
    pass
except FileNotFoundError:
    print(f"Error: spotdl command not found. Please install spotdl.")
except Exception as e:
    print(f"Error: {e}")

# Add a success message
print("Download complete!")

# <img src='https://upload.wikimedia.org/wikipedia/commons/5/5f/FFmpeg_Logo_new.svg' height="34"/>

### *Encoder*

In [ ]:
#@title <h4> *Normal Encoder*

import subprocess

input_file = "" #@param {type:"string"}
output_file = "" #@param {type:"string"}
arguments = "" #@param {type:"string"}

while True:
    command = f"ffmpeg -i {input_file} {arguments} {output_file}"
    process = subprocess.Popen(command, shell=True)
    process.wait()
    if process.returncode == 0:
        print("FFmpeg command executed successfully!")
        break
    else:
        print("FFmpeg command failed. Retrying...")

In [ ]:
#@title <h4> *Split & Encode*
import subprocess

input_file = "" #@param {type:"string"}

width = subprocess.check_output(['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=width', '-of', 'csv=p=0', input_file]).decode('utf-8').strip()
height = subprocess.check_output(['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=height', '-of', 'csv=p=0', input_file]).decode('utf-8').strip()

print("Processing video...")

if int(width) == 1280 and int(height) < 600:
    size = "-s 1280:538"
elif int(width) == 960 and int(height) < 500:
    size = "-s 960:458"
elif int(width) == 854 and int(height) < 400:
    size = "-s 854:356"
else:
    size = ""

segment_time = 600  # 10 minutes in seconds
segment_number = 1

while True:
    output_file = f"file{segment_number:02d}.mkv"
    cmd = f"ffmpeg -i {input_file} -c:v libx265 {size} -c:a copy -c:s copy -segment_time {segment_time} {output_file}"
    print(f"Processing: {output_file}")
    subprocess.run(cmd, shell=True)
    segment_number += 1

### *Subtitle Encoder*

In [ ]:
input1 = "" #@param {type:"string"}
input2 = "" #@param {type:"string"}
filename = "" #@param {type:"string"}
!ffmpeg -loglevel error -i "$input1" -i "$input2" -c copy -map_metadata -1 -map_chapters -1 drive/MyDrive/"$filename".mkv

### *Video Splitter Tool*

In [ ]:
input_file = "" #@param {type:"string"}
segment_time = "00:15:00" #@param {type:"string"}
output_dir = "drive/MyDrive/DriveFire/Segments"

import os
import subprocess

if os.path.isfile(input_file):
    os.makedirs(output_dir, exist_ok=True)
    try:
        subprocess.run([
            "ffmpeg", "-y", "-i", input_file,
            "-segment_time", segment_time,
            "-f", "segment", "-c", "copy",
            "-map_chapters", "-1", "-map_metadata", "-1",
            f"{output_dir}/file%02d.mkv"
        ], check=True)
        print(f"Output directory contents:")
        !ls {output_dir}
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg command failed with return code {e.returncode}.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("Input file does not exist.")

In [ ]:
!printf "file '%s'\n" drive/MyDrive/DriveFire/Vid/*.mkv | sort -V > list.txt

In [ ]:
!ffmpeg -hide_banner -y -f concat -safe 0 -i list.txt -c copy -map 0 -map_metadata -1 -map_chapters -1 "drive/MyDrive/Top Gun Maverick (2022).mkv"

In [ ]:
!cat list.txt

### *Multi-Image Generator*

In [ ]:
input_file = "" #@param {type:"string"}
seek_time = "" #@param {type:"string"}
num_of_frames = 10 #@param {type:"integer", default: 10}
increment = 1 #@param {type:"integer", default: 1}

import os
import glob
from IPython.display import Image, display, Markdown
import datetime

for filename in glob.glob('*.jpg'):
    os.remove(filename)

if ':' in seek_time:
    seek_time_dt = datetime.datetime.strptime(seek_time, "%H:%M:%S")
else:
    seek_time_dt = datetime.datetime(1900, 1, 1) + datetime.timedelta(seconds=int(seek_time))

for i in range(num_of_frames):
    timestamp_dt = seek_time_dt + datetime.timedelta(seconds=i*increment)
    timestamp = timestamp_dt.strftime("%H:%M:%S")
    command = f"ffmpeg -y -loglevel error -ss {timestamp} -i \"{input_file}\" -frames:v 1 /content/{timestamp}.jpg"
    os.system(command)

for filename in sorted(glob.glob('*.jpg'), key=lambda x: int(x[:-4].replace(':', ''))):
    display(Image(filename))
    display(Markdown(f"***{filename[:-4]}***"))

for filename in glob.glob('*.jpg'):
    os.remove(filename)

### *Image Generator*

In [ ]:
import os
import subprocess
from IPython.display import Image, display, Markdown

input_file = "" #@param {type:"string"}
seek_normal = "" #@param {type:"string"}
filter = "" #@param {type:"string"}
first_frame = False #@param {type:"boolean"}
last_frame = False #@param {type:"boolean"}

def extract_frame(input_file, output_file, ss, filter=""):
    try:
        if filter:
            !ffmpeg -loglevel error -y -ss {ss} -i "{input_file}" -vf "{filter}" -vframes 1 -q 0 {output_file}
        else:
            !ffmpeg -loglevel error -y -ss {ss} -i "{input_file}" -vframes 1 -q 0 {output_file}
    except:
        pass

def get_duration(input_file):
    try:
        return int(float(subprocess.check_output(f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 -i \"{input_file}\"", shell=True).decode("utf-8").strip()))
    except:
        return 0

def format_time(time_str):
    h, m, s = map(int, time_str.split(':'))
    return f"{h:02d}:{m:02d}:{s:02d}"

def get_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

if os.path.exists(input_file):
    if first_frame:
        extract_frame(input_file, "first.jpg", 0)
        if os.path.exists("first.jpg"):
            with open("first.jpg", "rb") as f:
                image_data = f.read()
            display(Image(data=image_data, format="jpg"))
            display(Markdown("***00:00:00*** &nbsp;-&nbsp; *First Frame*"))

    if last_frame:
        duration = get_duration(input_file)
        timestamp = duration - 1
        while True:
            try:
                extract_frame(input_file, "last.jpg", timestamp)
                break
            except:
                timestamp -= 1
                if timestamp < 0:
                    break
        if os.path.exists("last.jpg"):
            timestamp_str = format_time(f"{timestamp//3600:02d}:{(timestamp%3600)//60:02d}:{timestamp%60:02d}")
            with open("last.jpg", "rb") as f:
                image_data = f.read()
            display(Image(data=image_data, format="jpg"))
            display(Markdown(f"***{timestamp_str}*** &nbsp;-&nbsp; *Last Frame*"))

    if seek_normal:
        seconds = get_seconds(seek_normal)
        extract_frame(input_file, "normal.jpg", seconds, filter)
        timestamp = format_time(seek_normal)
        if os.path.exists("normal.jpg"):
            with open("normal.jpg", "rb") as f:
                image_data = f.read()
            display(Image(data=image_data, format="jpg"))
            display(Markdown(f"***{timestamp}*** &nbsp;-&nbsp; *Normal Frame*"))
        else:
            display(Markdown(f"***{timestamp}*** &nbsp;-&nbsp; *Normal Frame is not found*"))

!rm -rf *.jpg

### *Trimmer*

In [25]:
input = "" #@param {type:"string"}
output = "" #@param {type:"string"}
start = "" #@param {type:"string"}
end = "" #@param {type:"string"}
re_encode = False #@param {type:"boolean"}

if start:
    start_str = "-ss " + str(int(start)) if start.isdigit() else "-ss " + start
else:
    start_str = ""

if end:
    if not re_encode:
        end_str = "-t " + str(int(end)) if end.isdigit() else "-t " + end
    else:
        end_str = "-to " + str(int(end)) if end.isdigit() else "-to " + end
else:
    end_str = ""

if not re_encode:
    !ffmpeg -y -loglevel error -i "$input" {start_str} {dur} -avoid_negative_ts 1 -c copy "$output".mkv
else:
    !ffmpeg -y -loglevel error -i "$input" {start_str} {end_str} -crf 0 -c:a copy "$output".mkv

### *MediaInfo*

In [ ]:
import subprocess
import json
import os

input_file = "" #@param {type:"string"}

if not input_file:
    print("Error: No input file provided.")
else:
    if not os.path.exists(input_file):
        print("Error: Input file does not exist.")
    else:
        file_size_mb = os.path.getsize(input_file) / (1024 * 1024)
        print(f"File size: {file_size_mb / 1024 if file_size_mb > 1024 else file_size_mb:.2f} {'GB' if file_size_mb > 1024 else 'MB'}")

        try:
            ffprobe_output = subprocess.check_output(f"ffprobe -v error -show_streams -show_format -print_format json \"{input_file}\"", shell=True).decode("utf-8")
            media_info = json.loads(ffprobe_output)

            print("Media Information:")
            for stream in media_info['streams']:
                print(f"  {stream['codec_type'].capitalize()}:")
                print(f"    Codec: {stream['codec_name']}")
                if stream['codec_type'] == 'video':
                    print(f"    Resolution: {stream['width']}x{stream['height']}")
                elif stream['codec_type'] == 'audio':
                    print(f"    Channels: {stream['channel_layout']}")

            duration_output = subprocess.check_output(f"ffprobe -v error -show_entries format=duration -sexagesimal -of default=noprint_wrappers=1:nokey=1 -i \"{input_file}\"", shell=True).decode("utf-8").strip()
            print(f"Duration: {duration_output}")

        except subprocess.CalledProcessError:
            print("Error: Unable to get media information using ffprobe.")

# <img src='https://biplobsd.github.io/RLabClone/img/title_youtube-dl.png' height="38"/>

In [ ]:
YTDL=!(command -v "yt-dlp")
if not YTDL:
  !curl -s -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
  !chmod a+rx /usr/local/bin/yt-dlp
URL = "" #@param {type:"string"}
quality = '0' #@param ["bv[ext=mp4]+ba[ext=m4a]","0"] {allow-input: true}
output = "%(title)s.%(ext)s" #@param ["%(title)s.%(ext)s"] {allow-input: true}
!yt-dlp --no-warnings -q --progress -f {quality} -o "$output" "$URL"

# <font size=5> ✦ *Colab Still Alive*

In [ ]:
some_str = ' ' * 5120000000000

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button");
     btn.click()
     }

   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

clear_output()

# <font size=6> Scratchpad

In [ ]:
#@markdown <font size=4>***Demucs***
from IPython.display import clear_output
input = "" #@param{type:"string"}
DEMUCS=!(command -v "demucs")
if not DEMUCS:
  !python3 -m pip install -U demucs
  clear_output()
!demucs "$input"

In [ ]:
#@markdown <font size=4>***Auto Sub***

input_file = "" #@param {type:"string"}
output_file = "" #@param {type:"string"}
device = "cuda" #@param ["cuda","cpu"]
compute_type = "float16" #@param ["float16","int8"]

from IPython.display import clear_output
!pip install faster-whisper
clear_output()

from faster_whisper import WhisperModel

model_size = "large-v2"
model = WhisperModel(model_size, device, compute_type=compute_type)

segments, _ = model.transcribe(input_file, beam_size=50, language="ko", task="translate", word_timestamps=True, vad_filter=True)

with open(output_file, "w") as f:
    for i, segment in enumerate(segments):
        print(f"Transcribing segment {i+1}/{len(segments)}")
        for word in segment.words:
            f.write("[%.2fs -> %.2fs] %s\n" % (word.start, word.end, word.word))

In [ ]:
#@markdown <font size=4>***Google Translator***
!pip install googletrans
import googletrans
from googletrans import Translator  # Import the Translator class from the googletrans library

file = "" #@param {type:"string"}
translator = Translator()  # Create a Translator object
with open(file, 'r') as f:  # Open the file in read mode
    text = f.read()  # Read the contents of the file
translated = translator.translate(text, src='ko', dest='en')  # Translate the text
print(translated.text)  # Print the translated text

In [ ]:
file= "" #@param{type:"string"}
from google.colab import files
files.download(file)